# Data Portal adaptation demo

**This demo implements an analytical workflow that integrates...**
* Climate-hazard probability datasets linked from Google Earth Engine
* Interactivity
* An assessment of which hazards are important at any location
* Data outputs compatible with adaptation reporting under the CRF

**What to notice**
* Important hazards are listed by hazard category; a display in terms of climate indicators is forthcoming
* Current and future hazard magnitudes and probabilities are displayed in the table, the graph, and the map
* Define location of interest by placing a circle-marker, drawing a polygon, or importing a GeoJSON describing a point or polygon
* The demo has three sections; be sure to scroll all the way to the right


In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
import ee
service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'data-portal-adaptation.json')

In [3]:
# GEE authentication using service account key stored in local directory
ee.Initialize(credentials)

In [4]:
import geemap
from ipyleaflet import CircleMarker, Polygon, LayersControl, DrawControl, ZoomControl
from ipywidgets import interact, interactive, interact_manual, Layout, Accordion
import ipywidgets as widgets
import sys, asyncio, codecs, json
import numpy as np

In [5]:
# Create decorator to prevent events triggers while slider is being moved
# From https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [6]:
# HAZARD INFO

hazards = {
    'ds': {
        'keyname': 'dryspells',
        'short_displayname': 'DS',
        'displayname': 'dryspells',
        'min': 0,
        'max': 100,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 100,
        'bin_width': 1,
        'hazard_category': 'drought'
    },
    'maxdryspell': {
        'keyname': 'maxdryspell',
        'short_displayname': 'mxDS',
        'displayname': 'max dryspell duration',
        'min': 0,
        'max': 365,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 73,
        'bin_width': 5,
        'hazard_category': 'drought'
    },
    'mtt35': {
        'keyname': 'mtt35',
        'short_displayname': 'MTT35',
        'displayname': 'days temp above 35',
        'min': 0,
        'max': 300,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 74,
        'bin_width': 5,
        'hazard_category': 'heat'
    },
    'modlandslide': {
        'keyname': 'modlandslide',
        'short_displayname': 'LS',
        'displayname': 'days moderate landslide risk',
        'min': 0,
        'max': 300,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 60,
        'bin_width': 5,
        'hazard_category': 'landslide'
    },
    'totalprecip': {
         'keyname': 'totalprecip',
         'short_displayname': 'totPR',
         'displayname': 'annual total precipitation',
         'min': 0,
         'max': 8500,
         'use_greaterthan': True,
         'years': list(range(2020, 2100)),
         'num_bins': 85,
         'bin_width': 100,
         'hazard_category': 'precipitation'
    },
    'maxprecip': {
        'keyname': 'maxprecip',
        'short_displayname': 'mxPR',
         'displayname': 'maximum daily precipitation',
         'min': 0,
         'max': 4000,
         'use_greaterthan': True,
         'years': list(range(2020, 2100)),
         'num_bins': 80,
         'bin_width': 20,
         'hazard_category': 'precipitation'
    },
    'riverflood': {
         'keyname': 'riverflood',
        'short_displayname': 'riverFLD',
         'displayname': 'riverine inundation',
         'min': 0,
         'max': 5,
         'use_greaterthan': True,
         'years': [1980, 2030, 2050, 2080],
         'num_bins': 10,
         'bin_width': 5,
         'hazard_category': 'flooding'
    }
}

FLOOD_YEARS = [2030, 2050, 2080]

HAZARD_CATEGORIES = ['heat', 'drought', 'landslide', 'precipitation', ] #'flooding']

hazard_data = {hazard: {
    'probs': ee.ImageCollection('users/tedwongwri/dataportal/MAPprobs/{}'.format(hazard)),
    'tprobs': ee.ImageCollection('users/tedwongwri/dataportal/thresholdMAPprobs/{}'.format(hazard)),
    }
    for hazard in hazards.keys()
}

all_percentile_data = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/allhazards')
tmp = ee.List([])
for hazard_name in hazards.keys():
    if hazards[hazard_name]['hazard_category'] in HAZARD_CATEGORIES:
        tmp = tmp.add(all_percentile_data.filterMetadata('hazard', 'equals', hazard_name).select('{}_p95'.format(hazards[hazard_name]['keyname'])).first())
percentile_data = ee.ImageCollection(tmp).toBands().rename([i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES])

In [7]:
# DATA AND INITIAL SETTINGS

initial_threshold = 30
min_threshold = 5
max_threshold = 50
threshold_interval = 5
DEFAULT_CURRENTYEAR = 2022
currentYear = DEFAULT_CURRENTYEAR
futureYear = 2050
prev_threshold_current = initial_threshold
prev_threshold_future = initial_threshold
prev_futureYear = 2050

prev_layers = []

persistent_threshold_value = initial_threshold
persistent_ev_current = None
persistent_ev_future = None
persistent_threshold_value = initial_threshold
initial_coords = [50.85045, 4.34878]  # Brussels
current_geom = ee.Geometry.Point(initial_coords)
current_location_marker = None
ev_val = 'expected'
mode_is_ev = False

current_hazard = 'maxdryspell'

MAPprobs = hazard_data[current_hazard]['tprobs']
currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

def update_data():
    global MAPprobs
    global currentprobs
    global futureprobs
    MAPprobs = hazard_data[current_hazard]['tprobs']
    currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
    futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

In [8]:
# MAP

upload_button = widgets.FileUpload(
    accept = '.geojson',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple = False,  # True to accept multiple files upload else False
    description = 'GeoJSON'
)

Map = geemap.Map(center=initial_coords, zoom=3)
Map.add_basemap('HYBRID')

Map.clear_controls()
layer_control = LayersControl(position='topright')
Map.add_control(layer_control)
Map.add_control(ZoomControl(position='bottomright'))
draw_control = DrawControl(circle={}, polyline={}, rectangle={}, position='topleft')
Map.add_control(draw_control)

current_location_marker = CircleMarker(location=initial_coords, name="location of interest")
Map.add_layer(current_location_marker)

vizParamsC = {
  'bands': ['threshold_' + str(initial_threshold)],
  'min': 0,
  'max': 1,
  'palette': ['000000', '909000'],
  'opacity': 0.5
}
vizParamsF = {
  'bands': ['threshold_' + str(initial_threshold)],
  'min': 0,
  'max': 1,
  'palette': ['000000', 'ff0000'],
  'opacity': 0.5
}

def latlon_to_lonlat(coords):
# Converts EE coords (x, y) to ipyleaflet coords (y, x)
# Works for geojson coords
    if type(coords[0]) == float:
        return [coords[1], coords[0]]
    else:
        return [[[i[1],i[0]] for i in j] for j in coords]
    
def update_location(geom):
    global current_geom
    global current_location_marker
    Map.remove_layer(current_location_marker)
    draw_control.clear()
    new_geom = geom
    if new_geom['type'] == 'Point':
        current_geom = ee.Geometry.Point(new_geom['coordinates'])
        ipl_loc = latlon_to_lonlat(new_geom['coordinates'])
        current_location_marker = CircleMarker(location=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
    else:
        current_geom = ee.Geometry.Polygon(new_geom['coordinates'])
        current_location_marker = Polygon(locations=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
    update_percentiles()
    update_table()
    update_probplot()
        
def drawcontrol_update(e):
    update_location(draw_control.last_draw['geometry'])
    update_probplot()
    update_table()

@debounce(0.5)
def uploadbutton_update(e):
    f = upload_button.data[-1]
    file_content = json.loads(codecs.decode(f, encoding="utf-8"))
    update_location(file_content['features'][0]['geometry'])
    polygon_coords = current_geom.coordinates().getInfo()[0]
    exes = np.array([i[0] for i in polygon_coords])
    whys = np.array([i[1] for i in polygon_coords])
    Map.setCenter(lat=np.mean(whys), lon=np.mean(exes))
    Map.set_trait('zoom', 9)
    update_probplot()
    update_table()
'''    
@debounce(0.5)
def marker_update(e):
    debug_display.set_trait('value', str(e))
    update_probplot()
    update_table()
    update_percentiles()
'''

draw_control.observe(drawcontrol_update)
upload_button.observe(uploadbutton_update)

def update_map():
# Updates map w new prob layers according to futureYear and whichever thresholds are appropriate given mode_is_ev state
# Updates some globals
    global prev_futureYear
    global prev_threshold_current
    global prev_threshold_future
    global prev_layers
    
    for layername in prev_layers:
        Map.remove_ee_layer(name=layername)
        
    if mode_is_ev:
        c_ev, dummy = ev_prob(currentYear)
        f_ev, dummy = ev_prob(futureYear)
        c_threshold = round(c_ev / hazards[current_hazard]['bin_width']) * hazards[current_hazard]['bin_width']
        f_threshold = round(f_ev / hazards[current_hazard]['bin_width']) * hazards[current_hazard]['bin_width']
    else:
        c_threshold = threshold_slider.value
        f_threshold = threshold_slider.value
        
    new_visparamsC = {
      'bands': ['threshold_' + str(c_threshold)],
      'min': 0,
      'max': 1,
      'palette': ['000000', '909000'],
      'opacity': 0.5
    }
    new_visparamsF = {
      'bands': ['threshold_' + str(f_threshold)],
      'min': 0,
      'max': 1,
      'palette': ['000000', 'ff0000'],
      'opacity': 0.5
    }
    currentlayername = str(currentYear) + ' Prob({0} {1} {2})'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1], str(c_threshold))
    Map.add_ee_layer(currentprobs, new_visparamsC, currentlayername)
    futurelayername = str(futureYear) + ' Prob({0} {1} {2})'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1], str(f_threshold))
    Map.add_ee_layer(futureprobs, new_visparamsF, futurelayername)
    prev_layers = [currentlayername, futurelayername]

In [9]:
# HAZARD DISPLAY

def nearest_floodyear(year):
    diffs = [abs(year-i) for i in FLOOD_YEARS]
    min_diff = min(diffs)
    return FLOOD_YEARS[diffs.index(min_diff)]

def update_hazard(e):
    global current_hazard
    global currentYear
    global futureYear
    current_hazard = hazard_selector.value
    if hazards[current_hazard]['hazard_category'] == 'flooding':
        currentYear = 2030
        futureYear = nearest_floodyear(futureYear)
        futureyear_slider.value = futureYear
    else:
        currentYear = DEFAULT_CURRENTYEAR
    update_data()
    update_table()
    #update_map()
    update_probplot()
    update_sentencedisplay()
    update_thresholdslider()
    
hazard_selector = widgets.Dropdown(
    options=[(hazards[h]['displayname'], h) for h in hazards.keys()],
    value=current_hazard,
    description='Climate-hazard indicator',
    disabled=False,
)
hazard_display = widgets.HTML()

hazard_selector.observe(update_hazard)

In [10]:
# SELECTORS

@debounce(0.5)
def threshold_update(e):
    global persistent_threshold_value
    persistent_threshold_value = threshold_slider.value
    threshold_slider.description = '{0} {1} ___'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1] )
    update_table()
    update_map()
    
@debounce(0.5)
def futureyear_update(e):
    global futureYear
    global futureprobs
    global prev_futureYear
    global persistent_ev_future
    if hazards[current_hazard]['hazard_category'] == 'flooding':
        futureyear_slider.value = nearest_floodyear(futureyear_slider.value)
    futureYear = futureyear_slider.value
    update_data()
    #persistent_ev_future = get_ev(futureYear)
    
    update_probplot()
    update_table()
    update_map()
    update_percentiles()
    prev_futureYear = futureYear
      
@debounce(0.5)
def mode_update(e):
    global mode_is_ev
    global persistent_ev_current
    global persistent_ev_future
    mode_toggle.tooltips = ['What {} can you expect?'.format(hazards[current_hazard]['short_displayname']), 'How probable is a specific {} level?'.format(hazards[current_hazard]['short_displayname'])]
    mode_is_ev = mode_toggle.value == ev_val
    if mode_is_ev:
    #    if not persistent_ev_current:
    #        persistent_ev_current = get_ev(currentYear)
    #    if not persistent_ev_future:
    #        persistent_ev_future = get_ev(futureYear)
        threshold_slider.layout.visibility = 'hidden'
    else:
        threshold_slider.layout.visibility = 'visible'
    update_table()
    update_map()

futureyear_slider = widgets.IntSlider(
    value = futureYear,
    min = currentYear + 1,
    max = 2080,
    step = 1,
    description = 'Future year',
    continuous_update = False,
    disabled=False
)

threshold_slider = widgets.IntSlider(
    value=initial_threshold,
    min=0,
    max=50,
    step=5,
    description='{0} {1} ___'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1] ),
    continuous_update=False,
    disabled=False
)
threshold_slider.layout.visibility = ['visible', 'hidden'][mode_is_ev * 1]

def update_thresholdslider():
    threshold_slider.max = hazards[current_hazard]['bin_width'] * hazards[current_hazard]['num_bins']
    threshold_slider.step = hazards[current_hazard]['bin_width']
    threshold_slider.description='{0} {1} ___'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1] )

mode_toggle = widgets.ToggleButtons(
    options=[ev_val, 'set magnitude'],
    description='',
    #disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['What {} can you expect?'.format(hazards[current_hazard]['short_displayname']), 'How probable is a specific {} level?'.format(hazards[current_hazard]['short_displayname'])],
    value='set magnitude',
    disabled=False
)

table_display = widgets.HTML()

threshold_slider.observe(threshold_update)
futureyear_slider.observe(futureyear_update)
mode_toggle.observe(mode_update)

In [11]:
# SENTENCE DISPLAY

def update_sentencedisplay():
    html = "<table>"
    for h in hazards.keys():
        html += "<tr><td>You currently have {0} risk from {1}. This risk {2} by {3}.</td></tr>".format("high/med/low", hazards[h]['displayname'], "increases/decreases/does not change", str(futureYear))
    html += "</table>"
    sentence_display.set_trait('value', html)
    
sentence_display = widgets.HTML()

In [12]:
# TABLES

def renameBands(img):
# MAPprob files have bandnames like "threshold_5"
# Returns threshold vals like "05"
    new_bandnames = [i.split('_')[1] for i in img.bandNames().getInfo()]
    max_length = max(len(i) for i in new_bandnames)
    for i in range(len(new_bandnames)):
        item = new_bandnames[i]
        while len(item) < max_length:
            item = '0' + item
        new_bandnames[i] = item
    return img.rename(new_bandnames)

def CRFtext(probstring, prob):
    # Assigns label to numerical probability according to Box 4 (p. 44) of
    # https://www.globalcovenantofmayors.org/wp-content/uploads/2019/08/Data-TWG_Reporting-Framework_GUIDENCE-NOTE_FINAL.pdf
    if prob > 0.05:
        return 'high'
    elif prob > 0.005:
        return 'moderate'
    elif prob > 0.0005:
        return 'low'
    else:
        return 'do not know'
    
def changetext(before, after, inv):
    if before == after:
        return 'no change'
    chg = after > before
    if inv:
        chg = not chg
    return ['decrease', 'increase'][chg * 1]

def ev_prob(year):
# Plot expected value of hazard for given year
    probs = hazard_data[current_hazard]['probs'].filterMetadata('year', 'equals', year).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    dataprobs = np.array([float(i) for i in probs.values()]) / np.sum([float(i) for i in probs.values()])
    mag_range = [(j.split('-')[0], j.split('-')[1]) for j in [i.split('_')[1] for i in list(probs.keys())]]
    avg_mags = np.array([(float(i[0]) + float(i[1])) / 2 for i in mag_range])
    ev = np.dot(dataprobs, avg_mags) 
    for idx in range(len(mag_range)):
        mag_pair = mag_range[idx]
        if ev >= float(mag_pair[0]) and ev < float(mag_pair[1]):
            break
    key = list(probs.keys())[idx]
    prob = probs[key]
    return ev, prob
    
def update_table():
# Inserts output HTML into table_display widget
# Different HTML depending on whether mode_is_ev
    if mode_is_ev:
        current_ev, current_prob = ev_prob(currentYear)
        future_ev, future_prob = ev_prob(futureYear)
        evstring_current = '{:.1f} days'.format(current_ev)
        evstring_future = '{:.1f} days'.format(future_ev)
       
    else:
        current_probs = hazard_data[current_hazard]['tprobs'].filterMetadata('year', 'equals', currentYear).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
        future_probs = hazard_data[current_hazard]['tprobs'].filterMetadata('year', 'equals', futureYear).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
        for k in current_probs.keys():
            if current_probs[k] is None:
                current_probs[k] = 0
        for k in future_probs.keys():
            if future_probs[k] is None:
                future_probs[k] = 0
        thresholds = [float(i.split('_')[1]) for i in current_probs.keys()]
        
        for idx in range(len(thresholds)):
            mag_pair = (thresholds[idx], thresholds[idx] + hazards[current_hazard]['bin_width'])
            if persistent_threshold_value >= mag_pair[0] and persistent_threshold_value < mag_pair[1]:
                break
        key = list(current_probs.keys())[idx]    
        current_prob = current_probs[key]
        future_prob = future_probs[key]
    
    probstring_current = '{:.1f}%'.format(current_prob * 100)
    probstring_current += ' (' + CRFtext(probstring_current, current_prob) + ')'
    probstring_future = '{:.1f}%'.format(future_prob * 100)
    probstring_future += ' (' + CRFtext(probstring_future, future_prob) + ')'
    if current_prob > 0:
        ri_string_current = '{:.1f} years'.format(1.0/current_prob)  # recurrence interval
    else:
        ri_string_current = '(unprecedented event)'
    if future_prob > 0:
        ri_string_future = '{:.1f} years'.format(1.0/future_prob)
    else:
        ri_string_future = '(unprecedented event)'
    if mode_is_ev:
        newHtml = '<br /><b>What ' + hazards[current_hazard]['short_displayname'] + ' can you expect?</b><table style="width:600px; border: 1px solid black;"><tr style="border-bottom: 2px solid black; text-align: center; font-weight: bold"><td>year</td><td>magnitude</td><td>probability</td><td>recurrence interval</td><td>magnitude change</td><td>probability change</td></tr>'
        newHtml += '<tr style="padding: 3px; text-align: center"><td>' + str(currentYear) + '</td><td>' + evstring_current + '</td><td>' + probstring_current + '</td><td>' + ri_string_current + '</td><td rowspan="2">' + changetext(current_ev, future_ev, False) + '</td><td rowspan="2">' + changetext(current_prob, future_prob, False) + '</td></tr>'
        newHtml += '<tr style="padding: 3px; text-align: center"><td>' + str(futureYear) + '</td><td>' + evstring_future + '</td><td>' + probstring_future + '</td><td>' + ri_string_current + '</td></tr></table>'
    else:
        newHtml = '<br /><b>How probable is {0} {1} {2}?</b><table style="width:500px; border: 1px solid black;"><tr style="border-bottom: 2px solid black; text-align: center; font-weight: bold"><td>year</td><td>probability</td><td>recurrence interval</td><td>probability change</td></tr>'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1], str(persistent_threshold_value))
        newHtml += '<tr style="padding: 5px; text-align: center"><td>' + str(currentYear) + '</td><td style="text-align: center">' + probstring_current + '</td><td>' + ri_string_current + '</td><td rowspan="2">' + changetext(current_prob, future_prob, False) + '</td></tr>'
        newHtml += '<tr style="padding: 5px; text-align: center"><td>' + str(futureYear) + '</td><td style="text-align: center">' + probstring_future + '</td><td>' + ri_string_future + '</td></tr></table>'
    
    table_display.set_trait('value', newHtml)

In [13]:
# PLOT

def update_probplot():
# Replots plots of probability vs EHE threshold value
    probplot.canvas.draw()
    probplot.canvas.flush_events()
    global current_plot
    global future_plot
    if current_plot:
        dummy = current_plot.pop(0)
        dummy.remove()
    if future_plot:
        dummy = future_plot.pop(0)
        dummy.remove()
    current_data = renameBands(currentprobs).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    for k in current_data.keys():
        if current_data[k] is None:
            current_data[k] = 0
    current_plot = plt.plot(['{}d'.format(str(i)) for i in current_data.keys()], [100 * i for i in current_data.values()], label=str(currentYear), color='green')
    future_data = renameBands(futureprobs).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    for k in future_data.keys():
        if future_data[k] is None:
            future_data[k] = 0
    future_plot = plt.plot([str(i) + 'd' for i in future_data.keys()], [100 * i for i in future_data.values()], label=str(futureYear), color='lime')
    plt.xlabel('{0} {1} threshold'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1]))
    plt.legend()
    keys = list(current_data.keys())
    ticks = ['{}d'.format(str(keys[i])) for i in range(len(keys)) if i % (len(keys) // 10) == 0]
    plt.xticks(ticks)
    plt.xlim((ticks[0], ticks[-1]))
    
plt.ioff()

probplot = plt.figure(figsize=(5,4))
current_plot = None
future_plot = None
plt.xlabel('{0} {1} threshold'.format(hazards[current_hazard]['short_displayname'], ['≤', '≥'][hazards[current_hazard]['use_greaterthan'] * 1]))
plt.ylabel('Probability (%)')

plt.ion();

In [14]:
# PERCENTILES
# Looks up historical 95th percentile
# Finds prob that (max of current/future prob within AOI) >= p95
# Does this for each hazard and color codes display based on max-prob hazard within each hazard class

percentile_codes = {
    i: ('#ffffff', 'possible risk') if i < 50 else (('#ffff00', 'moderate risk') if i < 75 else (('#ff9900', 'high risk') if i < 90 else ('#ff0000', 'very high risk'))) for i in range(101)
}

def update_percentiles():
    local_percentiles = percentile_data.reduceRegion(ee.Reducer.max(), current_geom).getInfo()
    lookup_thresholds = {}
    for hn in [i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES]:
        mags = [i * hazards[hn]['bin_width'] for i in range(hazards[hn]['num_bins'] + 1)]
        diffs = [local_percentiles[hn] - i for i in mags]
        i = 0
        while i < len(diffs) - 2 and diffs[i] > 0:
            i += 1
        lookup_thresholds[hn] = mags[i]
    local_currentprobs = {
        hazard_name: hazard_data[hazard_name]['tprobs'].filterMetadata('year', 'equals', currentYear).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo() for hazard_name in [i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES]
    }
    local_futureprobs = {
        hazard_name: hazard_data[hazard_name]['tprobs'].filterMetadata('year', 'equals', futureYear).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo() for hazard_name in [i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES]
    }
    current_hazlevel = {
        hazcat: round(100 * max([[local_currentprobs[hazard_name]['threshold_{}'.format(str(lookup_thresholds[hazard_name]))], 0][int(lookup_thresholds[hazard_name]==0)] for hazard_name in [i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES] if hazards[hazard_name]['hazard_category']==hazcat])) for hazcat in HAZARD_CATEGORIES
    }
    future_hazlevel = {
        hazcat: round(100 * max([[local_futureprobs[hazard_name]['threshold_{}'.format(str(lookup_thresholds[hazard_name]))], 0][int(lookup_thresholds[hazard_name]==0)] for hazard_name in [i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES] if hazards[hazard_name]['hazard_category']==hazcat])) for hazcat in HAZARD_CATEGORIES
    }
    html = "<table style='width: 200px'><tr><td><b>{0}</b><td><b>{1}</td></tr>".format(str(currentYear), str(futureYear))
    for hazcat in HAZARD_CATEGORIES:
        html += "<tr><td style='padding-left: 2px; border: 1px solid gray; background-color:{0}'>{1}<br /><i style='font-size:9pt'>{2}</i></td>".format(percentile_codes[current_hazlevel[hazcat]][0], hazcat, percentile_codes[current_hazlevel[hazcat]][1])
        html += "<td style='padding-left: 2px; border: 1px solid gray; background-color:{0}'>{1}<br /><i style='font-size:9pt'>{2}</i></td></tr>".format(percentile_codes[future_hazlevel[hazcat]][0], hazcat, percentile_codes[future_hazlevel[hazcat]][1])
    html += "</table>"    
    hazard_display.set_trait('value', html)

In [15]:
local_percentiles = percentile_data.reduceRegion(ee.Reducer.max(), current_geom).getInfo()
lookup_thresholds = {}
for hn in [i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES]:
    mags = [i * hazards[hn]['bin_width'] for i in range(hazards[hn]['num_bins'] + 1)]
    diffs = [local_percentiles[hn] - i for i in mags]
    i = 0
    while i < len(diffs) - 2 and diffs[i] > 0:
        i += 1
    lookup_thresholds[hn] = mags[i]
local_currentprobs = {
    hazard_name: hazard_data[hazard_name]['tprobs'].filterMetadata('year', 'equals', currentYear).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo() for hazard_name in [i for i in hazards.keys() if hazards[i]['hazard_category'] in HAZARD_CATEGORIES]
}
local_currentprobs

{'ds': {'threshold_0': 1,
  'threshold_1': 1,
  'threshold_10': 0.702340034449753,
  'threshold_11': 0.702340034449753,
  'threshold_12': 0.702340034449753,
  'threshold_13': 0.702340034449753,
  'threshold_14': 0.6591488197541657,
  'threshold_15': 0.6591488197541657,
  'threshold_16': 0.6591488197541657,
  'threshold_17': 0.6591488197541657,
  'threshold_18': 0.6591488197541657,
  'threshold_19': 0.5941017574867608,
  'threshold_2': 1,
  'threshold_20': 0.5941017574867608,
  'threshold_21': 0.5873034849874946,
  'threshold_22': 0.5873034849874946,
  'threshold_23': 0.5863103792009575,
  'threshold_24': 0.5863103792009575,
  'threshold_25': 0.4174441713338996,
  'threshold_26': 0.3458228519879168,
  'threshold_27': 0.3171373524498517,
  'threshold_28': 0.2877287977478506,
  'threshold_29': 0.2231968785403437,
  'threshold_3': 0.9495295384543306,
  'threshold_30': 0.2231968785403437,
  'threshold_31': 0.06939060884627289,
  'threshold_32': 0.06939060884627289,
  'threshold_33': 0.06939

In [16]:
debug_display = widgets.HTML()

In [17]:
update_map()
update_percentiles()
update_probplot()
update_table()

In [18]:
# TAB LAYOUT
map_box = widgets.VBox([upload_button, Map], layout=Layout(width='600px', height='600px'))
hbar = widgets.HTML(value='<br/><div style="height:6px; background-color: #9090c0"></div>')
slider_box = widgets.VBox([futureyear_slider, threshold_slider], layout=Layout(align_items='flex-start'))
selector_box = widgets.VBox([hazard_selector, hbar, slider_box, hbar, mode_toggle], layout=Layout(width='400px', height='600px'))
plot_box = widgets.HBox([probplot.canvas])
tab_area = widgets.Tab(children=[hazard_display, table_display, plot_box], layout=Layout(width='600px', height='600px'))
for i in range(3):
    tab_area.set_title(i, ['Hazards', 'Probability table', 'Probability plot'][i])
dashboard = widgets.HBox([map_box, selector_box, tab_area], layout=Layout(width='1600px', height='600px'))

In [19]:
dashboard